In [ ]:
import faker, csv, random, itertools, seaborn, math

In [ ]:
fake = faker.Faker()

In [ ]:
with open("categories.csv") as file:
    categories = list(csv.DictReader(file))

In [ ]:
# Adding list of barcode to each category
faker.Faker.seed(0)
random.seed(0)
for cat in categories:
    cat["barcode_ean13"] = [fake.ean() for _ in range(1 + int(1000 * math.exp(-10*random.random())))]
seaborn.histplot([len(cat["barcode_ean13"]) for cat in categories])
print(set(len(cat["barcode_ean13"]) for cat in categories))

In [ ]:
# Check all barcodes are unique, if not change seeds and range_barcode
number_barcodes = sum(len(p["barcode_ean13"]) for p in categories)
uniques_barcodes = set(itertools.chain(*[p["barcode_ean13"] for p in categories]))
assert len(uniques_barcodes) == number_barcodes

In [ ]:
# create the product list
products = []
for cat in categories:
    for code in cat["barcode_ean13"]:
        products.append(
            {
                "product_category": cat["product_category"],
                "pdt_SUB_CATEGORY": cat["pdt_SUB_CATEGORY"],
                "barcode_ean13": code
            })
len(products)

In [ ]:
# Create a list of unique id for sites
faker.Faker.seed(1)
sites_id = list(set(fake.ean() for _ in range(500)))
len(sites_id)

In [ ]:
# assign a site group randomly for each site
random.seed(0)
sites = [{"site_key": site_id, "grp": random.choice([None, "hyp", "sup", "prx"]) } for site_id in sites_id]

In [ ]:
%%time
faker.Faker.seed(0)
random.seed(0)

data = []
for site in sites:
    
    prod_ids = random.sample(list(range(len(products))),  len(products)//20)
    for prod_id in prod_ids:
        prod = products[prod_id]
        data.append({
            "product_category": prod["product_category"],
            "pdt_SUB_CATEGORY": prod["pdt_SUB_CATEGORY"],
            "barcode_ean13": "f_" + prod["barcode_ean13"],
            "site_key": "f_" + site["site_key"],
            "grp": site["grp"],
            "DATE": str(fake.date_this_year()),
            "CA": round(1 + random.random()*50, 2)
          })
len(data)

In [ ]:
data[0]

In [ ]:
%%time

all = [
    {
        "product_category": prod["product_category"],
        "pdt_SUB_CATEGORY": prod["pdt_SUB_CATEGORY"],
        "barcode_ean13": "f_" + prod["barcode_ean13"],
        "site_key": "f_" + site["site_key"],
        "DATE": str(fake.date_this_year()),
        "CA": round(1 + random.random()*50, 2)
    }
    for r in data
]
print("all", len(all))
with open("data/crf_sales_group_data.csv", "w") as csvfile:
    writer = csv.DictWriter(csvfile, all[0].keys())
    writer.writeheader()
    writer.writerows(all)


for sub in ["hyp", "sup", "prx"]:
    dat = [
        {
            "product_category": prod["product_category"],
            "pdt_SUB_CATEGORY": prod["pdt_SUB_CATEGORY"],
            "barcode_ean13": "f_" + prod["barcode_ean13"],
            "site_key": "f_" + site["site_key"],
            "DATE": str(fake.date_this_year()),
            "CA": round(1 + random.random()*50, 2)
        }
        for r in data if r["grp"] == sub
    ]
    print(sub, len(dat))
    with open(f"data/crf_{sub}_sales_fr_data.csv", "w") as csvfile:
        writer = csv.DictWriter(csvfile, dat[0].keys())
        writer.writeheader()
        writer.writerows(dat)
